In [ ]:
import pandas as pd

# BeautifulSoup

In [ ]:
from bs4 import BeautifulSoup
import requests

## 현재 상영하고 있는 영화

In [ ]:
release = pd.DataFrame(columns = ['title','director', 'genre'])

In [ ]:
url = 'https://movie.naver.com/movie/running/current.nhn?order=reserve'
html = requests.get(url).content
soup = BeautifulSoup(html, 'lxml')

In [ ]:
# soup

In [ ]:
# li들 = 영화들
current_raw = soup.find('ul', 'lst_detail_t1').find_all('li')
current_raw[0]

In [ ]:
title = current_raw[0].img['alt']
genre = ", ".join(map(lambda x: x.text, current_raw[0].find_all('dd')[4]\
                                                    .find_all('span', 'link_txt')[0].find_all('a')))
director = current_raw[0].find_all('dd')[3].find_all('span', 'link_txt')[1].text.split(',')
director = ", ".join(map(lambda x: x.strip(), director))

In [ ]:
release = release.append({'title':title,'director':director, 'genre':genre}, ignore_index = True)

In [ ]:
for i in range(10):
    try:
        title = current_raw[i].img['alt']
        genre = ", ".join(map(lambda x: x.text, current_raw[i].find_all('dd')[4]\
                                                .find_all('span', 'link_txt')[0].find_all('a')))
        director = current_raw[i].find_all('dd')[3].find_all('span', 'link_txt')[1].text.split(',')
        director = ", ".join(map(lambda x: x.strip(), director))
    except:
        print(f"'{i+1}번째 상영작'을 크롤링하는 중에 오류가 발생하였습니다.")
        print(f"title: {title}\ngenre: {genre}\ndirector: {director}\n")

    finally:
        release = release.append({'title':title,'director':director, 'genre':genre}, 
                                           ignore_index = True)

# Selenium

In [ ]:
from selenium import webdriver

## 영화 검색

In [ ]:
url = "https://movie.naver.com"

driver = webdriver.Chrome(executable_path = '/Users/yejin/Desktop/chromedriver')
driver.get(url)
search = driver.find_element_by_css_selector('#ipt_tx_srch')
search.send_keys(input())
# time.sleep(1)
driver.implicitly_wait(0.5)

title = driver.find_element_by_css_selector('#jAutoMV > ul > li:nth-child(1)').get_attribute('data-title')
# code = driver.find_element_by_css_selector('#jAutoMV > ul > li:nth-child(1) > a').get_attribute('href')

In [ ]:
driver.find_element_by_css_selector('#jSearchArea > div > button').click()

In [ ]:
driver.find_element_by_css_selector('#old_content > ul.search_list_1 > li > dl > dt > a').click()

## 영화 리뷰 크롤링

In [ ]:
driver.find_element_by_css_selector('#movieEndTabMenu > li:nth-child(5) > a').click()

In [ ]:
driver.switch_to.frame("pointAfterListIframe")

In [ ]:
p = 2 # 페이지
driver.find_element_by_css_selector(f'a#pagerTagAnchor{p}').click()

In [ ]:
i = 1 # i번째 리뷰
star = int(driver.find_element_by_css_selector(f'body > div > div > div.score_result > ul > \
                                                li:nth-child({i}) > div.star_score > em').text)
review = driver.find_element_by_css_selector(f'#_filtered_ment_{i-1}').text

In [ ]:
star

In [ ]:
review